In [40]:
!pip install selenium

     |████████████████████████████████| 904 kB 1.2 MB/s eta 0:00:01


In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [55]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from urllib.request import urlretrieve
from tqdm import tqdm

def web_crawler(keyword):
    print("{} 이미지 크롤링".format(keyword))

    # 웹 접속 - 네이버 이미지 접속

    driver_path = os.getenv("HOME") +"/aiffel/selenium/chromedriver.ext"
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(30)

    url = 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query={}'.format(keyword)
    driver.get(url)

    #페이지 스크롤 다운
    body = driver.find_element_by_css_selector('body')
    for i in range(12):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    #이미지 링크 수집
    imgs = driver.find_elements_by_css_selector('img._img')
    result = []
    for img in tqdm(imgs):
        if 'http' in img.get_attribute('src'):
            result.append(img.get_attribute('src'))

    driver.close()

    #폴더생성
    if not os.path.isdir('./dataset/{}'.format(keyword)):
        os.mkdir('./dataset/{}'.format(keyword))

    #다운로드
    for index, link in tqdm(enumerate(result)):
        start = link.rfind('.')
        end = link.rfind('&')
        # print(link[start:end])
        filetype = link[start:end] #.png

        urlretrieve(link , './dataset/{}/{}{}{}'.format(keyword,keyword,index,filetype))
    time.sleep(3)
    

keyword_list = ["레드벨벳 아이린", "강호동"]

for keyword in keyword_list:
    web_crawler(keyword)

레드벨벳 아이린 이미지 크롤링


WebDriverException: Message: 'chromedriver.ext' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [58]:
pwd

'/home/ssac13/aiffel'

In [61]:
driver = webdriver.Chrome('../Desktop/selenium/chromedriver.exe')
driver.implicitly_wait(30)

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [57]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from urllib.request import urlretrieve
from tqdm import tqdm

def web_crawler(keyword):
    if not os.path.isdir('kpop_img/'): # 폴더 없으면 폴더 생성
        os.mkdir('kpop_img/')
    
    print("{} 이미지 크롤링".format(keyword))

    # 웹 접속 - 네이버 이미지 접속
    driver = webdriver.Chrome('./selenium/chromedriver.exe')
    driver.implicitly_wait(30)

    url = 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query={}'.format(keyword)
    driver.get(url)

    #페이지 스크롤 다운
    body = driver.find_element_by_css_selector('body')
    for i in range(12):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    #이미지 링크 수집
    imgs = driver.find_elements_by_css_selector('img._img')
    result = []
    for img in tqdm(imgs):
        if 'http' in img.get_attribute('src'):
            result.append(img.get_attribute('src'))

    driver.close()

    # 폴더 없으면 폴더 생성
    if not os.path.isdir('dataset/{}/'.format(keyword)): 
        os.mkdir('dataset/{}/'.format(keyword))

    #다운로드
    for index, link in tqdm(enumerate(result)):
        start = link.rfind('.')
        end = link.rfind('&')
        # print(link[start:end])
        filetype = link[start:end] #.png

        urlretrieve(link , 'dataset/{}/{}{}'.format(keyword,index,filetype))
    time.sleep(3)
    

keyword_list = ["레드벨벳 아이린", "장동건"]

for keyword in keyword_list:
    web_crawler(keyword)

레드벨벳 아이린 이미지 크롤링


WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [18]:
import dlib, cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageFont, ImageDraw, Image
import tensorflow.keras 
from tensorflow.keras import backend as K
import glob

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor('models/shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('models/dlib_face_recognition_resnet_model_v1.dat')

In [32]:
def find_faces(img):
    dets = detector(img, 1)

    if len(dets) == 0:
        return np.empty(0), np.empty(0), np.empty(0)
    
    rects, shapes = [], []
    shapes_np = np.zeros((len(dets), 68, 2), dtype=np.int)
    for k, d in enumerate(dets):
        rect = ((d.left(), d.top()), (d.right(), d.bottom()))
        rects.append(rect)

        shape = sp(img, d)
        
        # convert dlib shape to numpy array
        for i in range(0, 68):
            shapes_np[k][i] = (shape.part(i).x, shape.part(i).y)

        shapes.append(shape)
        
    return rects, shapes, shapes_np


def encode_faces(img, shapes):
    face_descriptors = []
    for shape in shapes:
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        face_descriptors.append(np.array(face_descriptor))

    return np.array(face_descriptors)

In [34]:
label_name = []
label_class = {}
img_paths = glob.glob("kpop_img/*")

for path in img_paths:
    name = path.split(".")[0][9:]
    label_name.append(name)
    label_class[name] = path

#print(label_name)
print(len(label_class))

5


In [35]:
def find_faces(img):
    dets = detector(img, 1)

    if len(dets) == 0:
        return np.empty(0), np.empty(0), np.empty(0)
    
    rects, shapes = [], []
    shapes_np = np.zeros((len(dets), 68, 2), dtype=np.int)
    for k, d in enumerate(dets):
        rect = ((d.left(), d.top()), (d.right(), d.bottom()))
        rects.append(rect)

        shape = sp(img, d)
        
        # convert dlib shape to numpy array
        for i in range(0, 68):
            shapes_np[k][i] = (shape.part(i).x, shape.part(i).y)

        shapes.append(shape)
        
    return rects, shapes, shapes_np


def encode_faces(img, shapes):
    face_descriptors = []
    for shape in shapes:
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        face_descriptors.append(np.array(face_descriptor))

    return np.array(face_descriptors)
    
    
def draw(img, comment, x, y, h, size):
    img = Image.fromarray(img)
    draw = ImageDraw.Draw(img)
    draw.text((x+5,y+h), comment, font=ImageFont.truetype("./batang.ttc", size), fill=(40,180,120))
    
    return np.array(img)  

In [36]:
# 데이터베이스 생성

label_name = []
label_class = {}
img_paths = glob.glob("models/dlib_face_recognition_resnet_model_v1.dat")

for path in img_paths:
    name = path.split(".")[0][9:]
    label_name.append(name)
    label_class[name] = path

#print(label_name)
print(len(label_class))

1


In [37]:
descs = []

for name, label_path in label_class.items():
    img = cv2.imread(label_path)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB
    _, img_shapes, _ = find_faces(img)
    descs.append([name, encode_faces(img, img_shapes)[0]])

np.save('descs.npy', descs)
print(descs)

TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: _dlib_pybind11.fhog_object_detector, image: array, upsample_num_times: int=0) -> _dlib_pybind11.rectangles

Invoked with: <_dlib_pybind11.fhog_object_detector object at 0x7fe7254af830>, None, 1

In [38]:
img = cv2.imread('models/image.jpg')


img = cv2.flip(img, 1) # 좌우 대칭
rects, shapes, _ = find_faces(img) # 얼굴 찾기
descriptors = encode_faces(img, shapes) # 인코딩

if(len(descriptors) == 0):
    print("얼굴 인식이 안되었네요ㅜㅜ \n다시 찍어주세요!")
elif(len(descriptors) > 1):
    print("여러 명이 인식이 되었네요ㅜㅜ \n혼자 다시 찍어주세요!")
else:
    desc = descriptors[0]
    x = rects[0][0][0] # 얼굴 X 좌표
    y = rects[0][0][1] # 얼굴 Y 좌표
    w = rects[0][1][1]-rects[0][0][1] # 얼굴 너비 
    h = rects[0][1][0]-rects[0][0][0] # 얼굴 높이        

    descs1 = sorted(descs, key=lambda x: np.linalg.norm([desc] - x[1]))
    dist = np.linalg.norm([desc] - descs1[0][1], axis=1)
    if dist < 0.5:
        name = descs1[0][0]
        comment = "{0}을 닮으셨네요. 올~~".format(name) 
        img = cv2.imread(label_class[name])
        img = cv2.resize(img, dsize=(200, 200))
        result = draw(img, comment, x-10, y, h, 36)
    else:
        comment = "닮은 연예인이 없네요\nㅜㅜ 성형하고 오세요!\n"
        img = cv2.imread("models/image.jpg")
        result = draw(img, comment, 50, 10, h, 24)

    print("거리: %.3f" % dist[0])
    print(comment)
#    cv2.imshow(name, result)
    result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB) # BGR -> RGB
    plt.imshow(result)

cv2.waitKey(0)
cv2.destroyAllWindows()

IndexError: list index out of range